In [1]:
import requests  
from lxml import etree 
import pickle
import os
from IPython.core.display import display, HTML
import timeit
import jieba

class MySearcherC9V0:
    """
    第七次课升级的搜索类版本：
    1、__init__()初始化过程加载自定义分词词典
    2、build_cache()改用cut_for_search进行分词
    3、search()对查询分词
    4、search()对分词结果取posting
    5、search()对posting lists进行合并(交集)
    6、build_cache()将posting保存格式改成只用doc_id(方便集合运算)
    7、rank()实现对候选文档打分排序
    8、score()实现对查询中包含的多词统计词频计分
    """
    def __init__(self, scale=1):
        self.docs = []
        self.load_data()
        self.docs *= scale
        self.cache = {}
        self.vocab = set()
        self.lower_preprocess()
        self.build_cache()
        jieba.load_userdict('dict.txt')
    
    def build_cache(self):
        doc_id = 0
        for doc in self.docs:
            doc_word_set = set()
            for word in jieba.cut_for_search(
                doc[3]
            ):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = set([result_item])
                    else:
                        self.cache[word].add(result_item)
                    self.vocab.add(word)
                    doc_word_set.add(word)
            doc_id += 1
    
    def search(self, query):
        result = None
        for keyword in jieba.cut(query.lower()):
            if keyword in self.cache:
                if result is None:
                    result = self.cache[keyword]
                else:
                    result = result & self.cache[keyword]
            else:
                result = set([])
                break
                
        if result is None:
            result = set([])
        
        sorted_result = self.rank(query, result)
        return sorted_result
                    
    def lower_preprocess(self):
        for doc_id in range(len(self.docs)):
            self.docs[doc_id].append(
                (self.docs[doc_id][1] 
                 + ' ' 
                 + self.docs[doc_id][2]).lower()
            )
    
    def simple_test(self):
        assert(len(self.search('tiktok')) > 1)
    
    def load_data(self):
        data_filename = 'news_list.dat'
        if os.path.exists(data_filename):
            with open(data_filename,'rb') as f:
                self.docs += pickle.load(f)
#                 self.docs = self.docs + pickle.load(f)
        else:
            url = 'http://news.163.com/special/0001386F/rank_tech.html'  
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36 Edg/85.0.564.63'}
            r = requests.get(url, headers=headers)  
            sel = etree.HTML(r.text) 
            link_set = set()
            news_list = []
            count = 0
            for item in sel.xpath('//td/a'):  
                title = item.text
                link = item.attrib['href']
            #     print(link, title)
                if link not in link_set:
                    r = requests.get(link, headers=headers)  
                    sel = etree.HTML(r.text)  
                    text_block = sel.xpath('//div[@id="endText"]') 
                #     print(''.join(text_block[0].itertext()))
                    if text_block:
                        content = ''.join(text_block[0].xpath('./p/text()'))
                        title = sel.xpath('//h1/text()')[0]
                        self.docs.append([link, title, content])
                    link_set.add(link)
                count += 1
                if count % 15 == 0:
                    print(count, 'processed.')
            with open(data_filename,'wb') as f:
                pickle.dump(self.docs, f)
    
    def highlight(self, text, keyword):
        idx = text.lower().find(keyword.lower())
        result = text
        if idx >= 0:
            ori_word = text[idx:idx+(len(keyword))]
            result = text.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
        return result
    
    def rank(self, query, result_set):
        result = []
        for doc_id in result_set:
            result.append([doc_id, 
                self.score(self.docs[doc_id],
                          query)])
        result.sort(key=lambda x: x[1], reverse=True)
        return result       
    
    def score(self, item, query):
        score = 0
        #todo cut
        for keyword in jieba.cut(query):
            score += item[1].lower().count(keyword.lower()) * 5 \
                      + item[2].lower().count(keyword.lower()) * 3
        return score
    
    def render_search_result(self, keyword):
        count = 0
        for item in self.search(keyword):
            count += 1
            display(HTML('{} [{}] {}'.format(count, item[1], 
                self.highlight(self.docs[item[0]][1], keyword) + '<br/>' + self.docs[item[0]][2])))

In [60]:
class MySearcherC9V1(MySearcherC9V0):
    """
    多关键词摘要和高亮
    """
    def highlight(self, item, query, sidelen=12):
        result = ''
        positions = []
        content_lower = item[2].lower()
        word_start_map = []
        word_end_map = []
        last_word_end = -1
        query_words = list(jieba.cut(query))
        for keyword in query_words:
            idx = content_lower.find(keyword.lower())
            positions.append(idx)

        for keyword in jieba.cut(content_lower):
            cur_word_start = last_word_end + 1
            cur_word_end = cur_word_start + len(keyword) - 1
            for i in range(cur_word_start, cur_word_end + 1):
                word_start_map.append(cur_word_start)
                word_end_map.append(cur_word_end)
            last_word_end = cur_word_end

    #     print(word_start_map)
    #     print(word_end_map)
    #     print(positions)
        positions.sort()
    #     print(positions)
        segments = []
        i = 0
        while i < len(positions):
            start_pos = max(positions[i] - sidelen, 0)
            end_pos = min(positions[i] + sidelen, len(content_lower) - 1)
            while (i < len(positions) - 1 
                and positions[i+1] - positions[i] <= 2 * sidelen):
                end_pos = min(positions[i+1] + sidelen, len(content_lower) - 1)
                i += 1  
            start_ddd = '...' if start_pos > 0 else ''
            end_ddd = '...' if end_pos < len(content_lower) else ''
#             print(len(content_lower), end_pos)
            segments.append(start_ddd 
                            + item[2][word_start_map[start_pos]:word_end_map[end_pos] + 1]
                            + end_ddd)
            i += 1
    #     print(segments)
        result = text = item[1] + '<br/>' + ''.join(segments)
        text_lower = text.lower()
        for keyword in query_words:
    #         print(keyword)
            idx = text_lower.find(keyword.lower())
            if idx >= 0:
                ori_word = text[idx:idx+(len(keyword))]
    #             print(ori_word)
                result = result.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
        return result
    
    def render_search_result(self, query):
        count = 0
        for item in self.search(query):
            count += 1
            display(HTML('{} [{}] {}'.format(count, item[1], 
                self.highlight(self.docs[item[0]], query))))
            print(self.highlight(self.docs[item[0]], query))

In [61]:
searcher_v1 = MySearcherC9V1()

In [62]:
searcher_v1.render_search_result('手机iPhone')

没大惊喜，但<span style="color:red";>iPhone</span>12这次诚意满满，我心动了<br/>...发布会，带来了姗姗来迟的<span style="color:red";>iPhone</span>12系列。虽然......华为超越苹果成为全球第二大<span style="color:red";>手机</span>厂商；然而，由于受到美国...


没大惊喜，但<span style="color:red";>iPhone</span>12这次诚意满满，我心动了<br/>...发布会，带来了姗姗来迟的<span style="color:red";>iPhone</span>12系列。虽然......华为超越苹果成为全球第二大<span style="color:red";>手机</span>厂商；然而，由于受到美国...


华为Mate40来了 硬刚<span style="color:red";>iPhone</span>12！买哪个？网友吵起来了<br/>...华为Mate40来了，硬刚<span style="color:red";>iPhone</span>12！该买哪一个......，华为Mate 40系列<span style="color:red";>手机</span>将与<span style="color:red";>iPhone</span> 12...


<span style="color:red";>iPhone</span> 12大猜想！直角边框、5G、蓝色、不送耳机<br/>...       （原标题：<span style="color:red";>iPhone</span> 12大猜想：......微博数码博主详细介绍了4款<span style="color:red";>手机</span>的具体参数和颜色：顶配...


<span style="color:red";>iPhone</span> 12系列摄影能力有多强？夜间拍照更强了<br/>...	作者：陈思学 梁桂海【<span style="color:red";>手机</span>中国新闻】介绍完<span style="color:red";>iPhone</span> 12后，苹果...


...

罗永浩回归！本人主持的秋季旧机发布会来了 与苹果撞期<br/>...直播之后，本人就暂时告别了<span style="color:red";>手机</span>发布会，不少人期待下一场......：罗永浩发布会时间与苹果<span style="color:red";>iPhone</span> 12发布日期...


苹果A14仿生芯片有多香？目前安卓类<span style="color:red";>手机</span>芯片看了都慌<br/>...非常关键的硬件，因为新款<span style="color:red";>iPhone</span> 12智能<span style="color:red";>手机</span>、首款使用苹果自主研发...


传苹果下周不会发布自研芯片Mac 11月才亮相<br/>...在很大程度上将集中在新款<span style="color:red";>iPhone</span> 12设备上，......发布会上似乎将重点放在智能<span style="color:red";>手机</span>和音频产品上，而在春季...


Win应用程序商店公开怼苹果：一视同仁，不屏蔽对手<br/>...的数量远远不及个人电脑和<span style="color:red";>手机</span>。游戏机的商业模式与围绕......商业模式完全不同，后者在<span style="color:red";>iPhone</span>和iPad设备...


特斯拉系统遭全球性宕机！有车主被困在沙漠两小时<br/>...表示，“特斯拉的App在<span style="color:red";>iPhone</span>上已经‘冻结’......系统的宕机，由于特斯拉的<span style="color:red";>手机</span>App正成为其生态系统...


In [38]:
def highlight(item, query, sidelen=12):
    result = ''
    positions = []
    content_lower = item[2].lower()
    word_start_map = []
    word_end_map = []
    last_word_end = -1
    query_words = list(jieba.cut(query))
    for keyword in query_words:
        idx = content_lower.find(keyword.lower())
        positions.append(idx)
        
    for keyword in jieba.cut(content_lower):
        cur_word_start = last_word_end + 1
        cur_word_end = cur_word_start + len(keyword) - 1
        for i in range(cur_word_start, cur_word_end + 1):
            word_start_map.append(cur_word_start)
            word_end_map.append(cur_word_end)
        last_word_end = cur_word_end
        
#     print(word_start_map)
#     print(word_end_map)
#     print(positions)
    positions.sort()
#     print(positions)
    segments = []
    i = 0
    while i < len(positions):
        start_pos = max(positions[i] - sidelen, 0)
        end_pos = min(positions[i] + sidelen, len(content_lower) - 1)
        while (i < len(positions) - 1 
            and positions[i+1] - positions[i] <= 2 * sidelen):
            end_pos = min(positions[i+1] + sidelen, len(content_lower) - 1)
            i += 1  
        start_ddd = '...' if start_pos > 0 else ''
        end_ddd = '...' if end_pos < len(content_lower) else ''
        print(len(content_lower), end_pos)
        segments.append(start_ddd 
                        + item[2][word_start_map[start_pos]:word_end_map[end_pos] + 1]
                        + end_ddd)
        i += 1
#     print(segments)
    result = text = item[1] + '<br/>' + ''.join(segments)
    text_lower = text.lower()
    for keyword in query_words:
#         print(keyword)
        idx = text_lower.find(keyword.lower())
        if idx >= 0:
            ori_word = text[idx:idx+(len(query))]
#             print(ori_word)
            result = result.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
    return result

display(HTML(highlight([
    '',
    '华为Mate40来了 硬刚iPhone12！买哪个？网友吵起来了', 
    '华为Mate40来了，硬刚iPhone12！该买哪一个？网友吵起来了） 每经编辑 何小桃这个10月，华为Mate 40系列手机将与iPhone 12系列手机正面对决'
], '对决华为系列手机')))

82 12
82 81


In [19]:
keywords = list(jieba.cut('华为Mate40来了 硬刚iPhone12！'))

In [20]:
keywords

['华为', 'Mate40', '来', '了', ' ', '硬刚', 'iPhone12', '！']

In [23]:
for keyword in keywords:
    print(keyword)

华为
Mate40
来
了
 
硬刚
iPhone12
！


In [29]:
for i in range(1,3+1):
    print(i)

1
2
3


In [ ]:
'abcdef'